## Spam Classfier
1. lets Fecth the data.
2. checking email structure
3. getting rid of html tags
4. email to plain text
5. email to word counter ( Replacing url , numbers , punctuations, converting to lowercase and stemming )
6. converting email counter to vector form .

In [3]:
import os
import tarfile
import urllib.request

DOWNLOAD_ROOT = "http://spamassassin.apache.org/old/publiccorpus/"
HAM_URL = DOWNLOAD_ROOT + "20030228_easy_ham.tar.bz2"
SPAM_URL = DOWNLOAD_ROOT + "20030228_spam.tar.bz2"
SPAM_PATH = os.path.join("datasets", "spam")

def fetch_spam_data(ham_url = HAM_URL, spam_url = SPAM_URL, spam_path = SPAM_PATH):
    # Check if there is the directory or no
    if not os.path.isdir(spam_path):
        # Create new if not
        os.makedirs(spam_path)
    for filename, url in (("ham.tar.bz2", ham_url), ('spam.tar.bz2', spam_url)):
        path = os.path.join(spam_path, filename)
        # if file is not available then download it.
        if not os.path.isfile(path):
            urllib.request.urlretrieve(url, path)
        # opening and extracting the file
        tar_bz2_file = tarfile.open(path)
        tar_bz2_file.extractall(path=spam_path)
        tar_bz2_file.close()



In [2]:
fetch_spam_data()

In [4]:
# loading all emails.
HAM_DIR = os.path.join(SPAM_PATH, 'easy_ham')
SPAM_DIR = os.path.join(SPAM_PATH, 'spam')
ham_filenames = [name for name in sorted(os.listdir(HAM_DIR)) if len(name) > 20]
spam_filenames = [name for name in sorted(os.listdir(SPAM_DIR)) if len(name) > 20]

In [5]:
len(ham_filenames), len(spam_filenames)

(2500, 500)

We can use Python's Email module to phrase these email ( this headers encoding, and so on).

In [44]:
import email
import email.policy

def load_email(is_spam, filename, spam_path = SPAM_PATH):
    directory = 'spam' if is_spam else 'easy_ham'
    with open(os.path.join(spam_path, directory, filename), 'rb') as f:
        return email.parser.BytesParser(policy=email.policy.default).parse(f)

In [10]:
# Opening and saving all email files in a list
ham_emails = [load_email(is_spam=False, filename=name) for name in ham_filenames]
spam_emails = [load_email(is_spam=True, filename=name) for name in spam_filenames]

In [14]:
# Checking value 
print(ham_emails[1].get_content().strip())

Martin A posted:
Tassos Papadopoulos, the Greek sculptor behind the plan, judged that the
 limestone of Mount Kerdylio, 70 miles east of Salonika and not far from the
 Mount Athos monastic community, was ideal for the patriotic sculpture. 
 
 As well as Alexander's granite features, 240 ft high and 170 ft wide, a
 museum, a restored amphitheatre and car park for admiring crowds are
planned
---------------------
So is this mountain limestone or granite?
If it's limestone, it'll weather pretty fast.

------------------------ Yahoo! Groups Sponsor ---------------------~-->
4 DVDs Free +s&p Join Now
http://us.click.yahoo.com/pt6YBB/NXiEAA/mG3HAA/7gSolB/TM
---------------------------------------------------------------------~->

To unsubscribe from this group, send an email to:
forteana-unsubscribe@egroups.com

 

Your use of Yahoo! Groups is subject to http://docs.yahoo.com/info/terms/



Some emails are actually multipart, with images and attachments (which can have their own attachments). Let's look at the various types of structures we have:

In [15]:
def get_email_structure(email):
    if isinstance(email, str):
        return email
    payload = email.get_payload()
    if isinstance(payload, list):
        return 'multipart({})'.format(','.join([get_email_structure(sub_email) for sub_email in payload]))
    else:
        return email.get_content_type()

In [16]:
from collections import Counter

def structures_counter(emails):
    structures = Counter()
    for email in emails:
        structure = get_email_structure(email)
        structures[structure] += 1
    return structures

In [19]:
structures_counter(ham_emails).most_common()

[('text/plain', 2408),
 ('multipart(text/plain,application/pgp-signature)', 66),
 ('multipart(text/plain,text/html)', 8),
 ('multipart(text/plain,text/plain)', 4),
 ('multipart(text/plain)', 3),
 ('multipart(text/plain,application/octet-stream)', 2),
 ('multipart(text/plain,text/enriched)', 1),
 ('multipart(text/plain,application/ms-tnef,text/plain)', 1),
 ('multipart(multipart(text/plain,text/plain,text/plain),application/pgp-signature)',
  1),
 ('multipart(text/plain,video/mng)', 1),
 ('multipart(text/plain,multipart(text/plain))', 1),
 ('multipart(text/plain,application/x-pkcs7-signature)', 1),
 ('multipart(text/plain,multipart(text/plain,text/plain),text/rfc822-headers)',
  1),
 ('multipart(text/plain,multipart(text/plain,text/plain),multipart(multipart(text/plain,application/x-pkcs7-signature)))',
  1),
 ('multipart(text/plain,application/x-java-applet)', 1)]

In [20]:
structures_counter(spam_emails).most_common()

[('text/plain', 218),
 ('text/html', 183),
 ('multipart(text/plain,text/html)', 45),
 ('multipart(text/html)', 20),
 ('multipart(text/plain)', 19),
 ('multipart(multipart(text/html))', 5),
 ('multipart(text/plain,image/jpeg)', 3),
 ('multipart(text/html,application/octet-stream)', 2),
 ('multipart(text/plain,application/octet-stream)', 1),
 ('multipart(text/html,text/plain)', 1),
 ('multipart(multipart(text/html),application/octet-stream,image/jpeg)', 1),
 ('multipart(multipart(text/plain,text/html),image/gif)', 1),
 ('multipart/alternative', 1)]

It seems that the ham emails are more often plain text, while spam has quite a lot of HTML. Moreover, quite a few ham emails are signed using PGP, while no spam is. In short, it seems that the email structure is useful information to have.

Looking at header now. 

In [21]:
for header , value in spam_emails[0].items():
    print(header,':', value)

Return-Path : <12a1mailbot1@web.de>
Delivered-To : zzzz@localhost.spamassassin.taint.org
Received : from localhost (localhost [127.0.0.1])	by phobos.labs.spamassassin.taint.org (Postfix) with ESMTP id 136B943C32	for <zzzz@localhost>; Thu, 22 Aug 2002 08:17:21 -0400 (EDT)
Received : from mail.webnote.net [193.120.211.219]	by localhost with POP3 (fetchmail-5.9.0)	for zzzz@localhost (single-drop); Thu, 22 Aug 2002 13:17:21 +0100 (IST)
Received : from dd_it7 ([210.97.77.167])	by webnote.net (8.9.3/8.9.3) with ESMTP id NAA04623	for <zzzz@spamassassin.taint.org>; Thu, 22 Aug 2002 13:09:41 +0100
From : 12a1mailbot1@web.de
Received : from r-smtp.korea.com - 203.122.2.197 by dd_it7  with Microsoft SMTPSVC(5.5.1775.675.6);	 Sat, 24 Aug 2002 09:42:10 +0900
To : dcek1a1@netsgo.com
Subject : Life Insurance - Why Pay More?
Date : Wed, 21 Aug 2002 20:31:57 -1600
MIME-Version : 1.0
Message-ID : <0103c1042001882DD_IT7@dd_it7>
Content-Type : text/html; charset="iso-8859-1"
Content-Transfer-Encoding : qu

There's probably a lot of useful information in there, such as the sender's email address (12a1mailbot1@web.de looks fishy), but we will just focus on the Subject header:

In [22]:
spam_emails[0]['subject']

'Life Insurance - Why Pay More?'

Splitting data into trainng and test

In [23]:
import numpy as np
from sklearn.model_selection import train_test_split

x = np.array(ham_emails + spam_emails, dtype=object)
y = np.array([0] * len(ham_emails) + [1] * len(spam_emails))

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=42)

In [61]:
# Getting rid of html tags
import re
from html import unescape

def html_to_plain_text(html):
    text = re.sub('<head.*?>.*?</head>', '', html, flags=re.M | re.S | re.I)
    text = re.sub('<a\s.*?>', ' HYPERLINK ', text, flags=re.M | re.S | re.I)
    text = re.sub('<.*?>', '', text, flags=re.M | re.S)
    text = re.sub(r'(\s*\n)+', '\n', text, flags=re.M | re.S)
    return unescape(text)

In [53]:
# HTML SPAMS
html_spams_emails = [email for email in x_train[y_train==1] if get_email_structure(email) == 'text/html']

sample_html_spam = html_spams_emails[7]

print(sample_html_spam.get_content().strip()[:1000],'....')

<HTML><HEAD><TITLE></TITLE><META http-equiv="Content-Type" content="text/html; charset=windows-1252"><STYLE>A:link {TEX-DECORATION: none}A:active {TEXT-DECORATION: none}A:visited {TEXT-DECORATION: none}A:hover {COLOR: #0033ff; TEXT-DECORATION: underline}</STYLE><META content="MSHTML 6.00.2713.1100" name="GENERATOR"></HEAD>
<BODY text="#000000" vLink="#0033ff" link="#0033ff" bgColor="#CCCC99"><TABLE borderColor="#660000" cellSpacing="0" cellPadding="0" border="0" width="100%"><TR><TD bgColor="#CCCC99" valign="top" colspan="2" height="27">
<font size="6" face="Arial, Helvetica, sans-serif" color="#660000">
<b>OTC</b></font></TD></TR><TR><TD height="2" bgcolor="#6a694f">
<font size="5" face="Times New Roman, Times, serif" color="#FFFFFF">
<b>&nbsp;Newsletter</b></font></TD><TD height="2" bgcolor="#6a694f"><div align="right"><font color="#FFFFFF">
<b>Discover Tomorrow's Winners&nbsp;</b></font></div></TD></TR><TR><TD height="25" colspan="2" bgcolor="#CCCC99"><table width="100%" border="0" 

In [62]:
html_to_plain_text(sample_html_spam.get_content().strip())[:1000]

'\nOTC\n\xa0Newsletter\nDiscover Tomorrow\'s Winners\xa0\nFor Immediate Release\nCal-Bay (Stock Symbol: CBYI)\nWatch for analyst "Strong Buy Recommendations" and several advisory newsletters picking CBYI.  CBYI has filed to be traded on the OTCBB, share prices historically INCREASE when companies get listed on this larger trading exchange. CBYI is trading around 25 cents and should skyrocket to $2.66 - $3.25 a share in the near future.\nPut CBYI on your watch list, acquire a position TODAY.\nREASONS TO INVEST IN CBYI\nA profitable company and is on track to beat ALL earnings estimates!\nOne of the FASTEST growing distributors in environmental & safety equipment instruments.\nExcellent management team, several EXCLUSIVE contracts.  IMPRESSIVE client list including the U.S. Air Force, Anheuser-Busch, Chevron Refining and Mitsubishi Heavy Industries, GE-Energy & Environmental Research.\nRAPIDLY GROWING INDUSTRY\nIndustry revenues exceed $900 million, estimates indicate that there could be

In [95]:
# Funtion to convert email to plain/text
def email_to_text(email):
    html = None
    for part in email.walk():
        ctype = part.get_content_type()
        if not ctype in ("text/plain", "text/html"):
            continue
        try:
            content = part.get_content()
        except: # in case of encoding issues
            content = str(part.get_payload())
        if ctype == "text/plain":
            return content
        else:
            html = content
    if html:
        return html_to_plain_text(html)

In [96]:
print(email_to_text(sample_html_spam))


OTC
 Newsletter
Discover Tomorrow's Winners 
For Immediate Release
Cal-Bay (Stock Symbol: CBYI)
Watch for analyst "Strong Buy Recommendations" and several advisory newsletters picking CBYI.  CBYI has filed to be traded on the OTCBB, share prices historically INCREASE when companies get listed on this larger trading exchange. CBYI is trading around 25 cents and should skyrocket to $2.66 - $3.25 a share in the near future.
Put CBYI on your watch list, acquire a position TODAY.
REASONS TO INVEST IN CBYI
A profitable company and is on track to beat ALL earnings estimates!
One of the FASTEST growing distributors in environmental & safety equipment instruments.
Excellent management team, several EXCLUSIVE contracts.  IMPRESSIVE client list including the U.S. Air Force, Anheuser-Busch, Chevron Refining and Mitsubishi Heavy Industries, GE-Energy & Environmental Research.
RAPIDLY GROWING INDUSTRY
Industry revenues exceed $900 million, estimates indicate that there could be as much as $25 billi

In [97]:
! pip install nltk

You should consider upgrading via the 'c:\users\navik\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [98]:
try:
    import nltk

    stemmer = nltk.PorterStemmer()
    for word in ('Computations', 'Computation','Computing','Computed','Compute','Compulsive'):
        print(word,'=>',stemmer.stem(word))
except ImportError:
    print('Error stemming requires the NLTK module.')
    stemmer =  None

Computations => comput
Computation => comput
Computing => comput
Computed => comput
Compute => comput
Compulsive => compuls


We will also ned a way to replace URLs with the word 'URL'. For this, w could us hard code reglar expression but we will just use th urlextract library.

In [99]:
pip install urlextract

Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the 'C:\Users\navik\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [100]:
try:
    import urlextract 

    url_extractor = urlextract.URLExtract()
    print(url_extractor.find_urls('Will it detect github.com and https://youyu.be/7Pq-S557XQU?t=3m32s'))
except ImportError:
    print('Error: replacing URLS requires the urlextract module.')
    url_extractor = None

['github.com', 'https://youyu.be/7Pq-S557XQU?t=3m32s']


#

In [101]:
from sklearn.base import BaseEstimator, TransformerMixin

# Defining class for transoformation of data
class EmailToWordCounterTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, strip_headers=True, lower_case = True, remove_punctuation = True, replace_urls=True, replace_numbers=True, stemming=True):

        self.strip_headers = strip_headers
        self.lower_case = lower_case
        self.remove_punctuation = remove_punctuation
        self.replace_urls = replace_urls
        self.replace_numbers = replace_numbers
        self.stemming = stemming

    def fit(self, X, Y =None):
        return self
    def transform(self, X, Y=None):
        X_transformed = [] # variable to save transofrmed data
        for email in X: # FOr each email in X
            text = email_to_text(email) or '' # converting email to text
            if self.lower_case: # if lower case transofromation is true
                text = text.lower()
            if self.replace_urls and url_extractor is not None: # if replacing url is true
                urls = list(set(url_extractor.find_urls(text))) # creating list of urls from extracted url
                urls.sort(key=lambda url:len(url), reverse=True) # sorting urls
                for url in urls: # for each extracted url replace it with ' URL '
                    text = text.replace(url,' URL ')
            if self.replace_numbers: # if replacing number is true
                text = re.sub(r'\d+(?:\.\d*)?(?:[eE][+-]?\d+)?','NUMBER',text) # regular expression to replace number
            if self.remove_punctuation: # if punction need to removed
                text = re.sub(r'\W+',' ', text,flags=re.M) # regular expression for removing puncuations
            word_counts = Counter(text.split()) # creating word counters
            if self.stemming and stemmer is not None: # stemming word counting same word as one
                stemmed_word_counts = Counter()
                for word, count in word_counts.items():
                    stemmed_word = stemmer.stem(word)
                    stemmed_word_counts[stemmed_word] +=count
                word_counts = stemmed_word_counts
            X_transformed.append(word_counts)
        return np.array(X_transformed)

Let's try this transoformation

In [102]:
X_few = x_train[:3]
x_few_wordcounts = EmailToWordCounterTransformer().fit_transform(X_few)
x_few_wordcounts

array([Counter({'chuck': 1, 'murcko': 1, 'wrote': 1, 'stuff': 1, 'yawn': 1, 'r': 1}),
       Counter({'the': 11, 'of': 9, 'and': 8, 'all': 3, 'christian': 3, 'to': 3, 'by': 3, 'jefferson': 2, 'i': 2, 'have': 2, 'superstit': 2, 'one': 2, 'on': 2, 'been': 2, 'ha': 2, 'half': 2, 'rogueri': 2, 'teach': 2, 'jesu': 2, 'some': 1, 'interest': 1, 'quot': 1, 'url': 1, 'thoma': 1, 'examin': 1, 'known': 1, 'word': 1, 'do': 1, 'not': 1, 'find': 1, 'in': 1, 'our': 1, 'particular': 1, 'redeem': 1, 'featur': 1, 'they': 1, 'are': 1, 'alik': 1, 'found': 1, 'fabl': 1, 'mytholog': 1, 'million': 1, 'innoc': 1, 'men': 1, 'women': 1, 'children': 1, 'sinc': 1, 'introduct': 1, 'burnt': 1, 'tortur': 1, 'fine': 1, 'imprison': 1, 'what': 1, 'effect': 1, 'thi': 1, 'coercion': 1, 'make': 1, 'world': 1, 'fool': 1, 'other': 1, 'hypocrit': 1, 'support': 1, 'error': 1, 'over': 1, 'earth': 1, 'six': 1, 'histor': 1, 'american': 1, 'john': 1, 'e': 1, 'remsburg': 1, 'letter': 1, 'william': 1, 'short': 1, 'again': 1, 'becom

In [103]:
# converting word counts into vector
from scipy.sparse import csr_matrix
class WordCounterToVectorTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, vocabulary_size=1000):
        self.vocabulary_size = vocabulary_size
    def fit(self, X, Y=None):
        total_count = Counter()
        for word_count in X:
            for word, count in word_count.items():
                total_count[word] += min(count, 10)
        most_common = total_count.most_common()[:self.vocabulary_size]
        self.vocabulary_ = {word: index+1 for index, (word, count) in enumerate(most_common)}
        return self
    
    def transform(self, X, Y=None):
        rows = []
        cols = []
        data = []

        for row, word_count in enumerate(X):
            for word, count in word_count.items():
                rows.append(row)
                cols.append(self.vocabulary_.get(word, 0))
                data.append(count)
        return csr_matrix((data, (rows,cols)), shape=(len(X),self.vocabulary_size + 1))

In [104]:
vocab_transformer = WordCounterToVectorTransformer(vocabulary_size=10)
X_few_vectors = vocab_transformer.fit_transform(x_few_wordcounts)
X_few_vectors

<3x11 sparse matrix of type '<class 'numpy.float64'>'
	with 20 stored elements in Compressed Sparse Row format>

In [105]:
X_few_vectors.toarray()

array([[ 6.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [99., 11.,  9.,  8.,  3.,  1.,  3.,  1.,  3.,  2.,  3.],
       [67.,  0.,  1.,  2.,  3.,  4.,  1.,  2.,  0.,  1.,  0.]])

In [106]:
vocab_transformer.vocabulary_

{'the': 1,
 'of': 2,
 'and': 3,
 'to': 4,
 'url': 5,
 'all': 6,
 'in': 7,
 'christian': 8,
 'on': 9,
 'by': 10}

In [107]:
from sklearn.pipeline import Pipeline

preprocess_pipeline = Pipeline([('email_to_wordcount',EmailToWordCounterTransformer()),
('wordcount_to_vctor', WordCounterToVectorTransformer()),])

x_train_transformed = preprocess_pipeline.fit_transform(x_train)

In [109]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

log_clf = LogisticRegression(solver='lbfgs', max_iter=1000, random_state=42)
score = cross_val_score(log_clf, x_train_transformed, y_train, cv=3, verbose=3)
score.mean()

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV] END ................................ score: (test=0.981) total time=   0.4s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s
[CV] END ................................ score: (test=0.984) total time=   0.6s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.2s remaining:    0.0s
[CV] END ................................ score: (test=0.991) total time=   0.7s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    2.0s finished


0.9854166666666666

In [111]:
from sklearn.metrics import precision_score, recall_score

x_test_transformed = preprocess_pipeline.transform(x_test)

log_clf.fit(x_train_transformed, y_train)

y_predict = log_clf.predict(x_test_transformed)

print('Precision: {:.2f}%'.format(100*precision_score(y_test,y_predict)))
print('Recall: {:.2f}%'.format( 100*recall_score(y_test,y_predict)))

Precision: 95.88%
Recall: 97.89%
